In [10]:
import zipfile
zipfile.ZipFile('train.zip').extractall()

In [1]:
from fastaudio.all import *
from fastai.vision.all import *

In [2]:
labels = pd.read_csv('train/targets.tsv', sep='\t', names=['path', 'gender'])

In [93]:
labels.path = 'train/'+labels.path

In [4]:
aud2spec = AudioToSpec.from_cfg(AudioConfig.Voice(f_min=0))

In [5]:
item_tfms = [RemoveSilence(), ResizeSignal(2000), aud2spec]

In [95]:
def get_x(row):
    return row.path + '.wav'
def get_y(row):
    return row.gender

In [96]:
audio_dblock = DataBlock(blocks=[AudioBlock, CategoryBlock],
                         item_tfms = item_tfms,
                         get_x = get_x, get_y = get_y
)

In [97]:
dls = audio_dblock.dataloaders(labels)

In [18]:
model =  xresnet18(True, n_out=2)

In [19]:
model[0][0].weight[:, [0]].shape

In [20]:
layer = model[0][0]
layer.in_channels = 1
layer.weight = nn.Parameter(layer.weight[:, [1]])
model[0][0] = layer

In [21]:
learn = Learner(dls, model, metrics=[accuracy])

In [22]:
learn.loss_func

In [23]:
learn.lr_find()

In [24]:
learn.fit_one_cycle(5, 1e-2)

In [25]:
import zipfile
zipfile.ZipFile('test.zip').extractall()

In [29]:
!cd test && ls | sed -e 's/\.wav$//' > ../targets.tsv

In [85]:
L(test_df.index.values.tolist())

(#3413) [0,1,2,3,4,5,6,7,8,9...]

In [86]:
test_audio_dblock = DataBlock(blocks=[AudioBlock],
                         item_tfms = item_tfms,
                         get_x = lambda x: f'test/{x.path}.wav',
                        splitter=lambda y: (L(y.index.values.tolist()),))

In [100]:
test_df = pd.read_csv('targets.tsv', sep='\t', names=['path'])
test_df.path = 'test/'+test_df.path
test_dl = dls.test_dl(test_df)

In [106]:
preds = learn.tta(dl = test_dl)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f6b8c7098b0>
Exception ignored in: Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f6b8c7098b0>
  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    Traceback (most recent call last):
self._shutdown_workers()  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__

    self._shutdown_workers()
  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    

In [105]:
preds

(tensor([[8.9336e-06, 9.9999e-01],
         [7.7786e-07, 1.0000e+00],
         [4.6656e-06, 1.0000e+00],
         ...,
         [9.7058e-01, 2.9418e-02],
         [9.9975e-01, 2.5037e-04],
         [9.9352e-01, 6.4787e-03]]),
 None,
 tensor([1, 1, 1,  ..., 0, 0, 0]))

In [138]:
preds[0][0].argmax().item()

1

In [141]:
pred_id = [i.argmax().item() for i in preds[0]]

In [159]:
save = pd.read_csv('targets.tsv', sep='\t', names=[])

In [158]:
save

""


In [162]:
save.loc[:, 1] = pred_id

In [163]:
len(pred_id), len(save)

(3413, 3413)

In [164]:
save.to_csv('preds.tsv', sep='\t', header=False)

In [161]:
save

""
00100026dbdffcd01cde6ee9b9a9d273
0014278276a6cc05fe8c522af0a677df
0026e20266ceba9cdda2c116e89d2f3b
008b37fc832d3dfad9105961c5801c02
00b1d7d0d6fdb25050041f6e2ae2871e
...
ffbea6a332d113f0bd16cf0b4ed07dc6
ffee25334e0247dd07d80d0eb3ce7886
fff86627949b37d236237ff895ff4268
fff8aa9bac38e85734f64aad2b2f3814
